In [1]:
from source.util import utils as u
from source import metrics, plots
from source import handshake, handshake2, scargc, hs
import sys
import time
import os
import psutil
import resource
import numpy as np
from sklearn.metrics import accuracy_score, matthews_corrcoef, f1_score

In [2]:
def doTrain(data, label, step, n_features, percent, epsilon, n_components, k):

    label_ = label
    
#     print(len(label))

    size = len(data)
    mod = size % 10

    if mod != 0:
        data = data[0:(size - mod), :]
        label = label[0:(size - mod)]

    tam = int((size - mod)/10)
    data = np.vsplit(data, 2)
    label = u.chunkify(label, 2)
    label_ = label_[0:(size-mod)]

    acc_percent = []
    f1_percent = []
    mcc_percent = []
    predicted = []
    updt = []
#     print(label[0])
    pool_size = int( len(data[0])/100 * percent )
    score = 0.0
    f1 = 0.0
    mcc = 0.0
    
    if pool_size >= k:
        pred, updt, gmm = hs.handshake2(data, label_, data[0], label[0], data[1], label[1], n_components, n_features, epsilon, percent, k)
        pred = np.array(pred, dtype=int)
        pred = pred.flatten()
        predicted = pred
        acc_percent, f1_percent, mcc_percent = metrics.makeBatches(label[0], predicted, len(label[0]), step)

        # sys.exit(0)

        for i in range(1, len(data)-1):
            print(i)
            pred, upd, gmm2 = hs.handshake2(data, label_, data[i], label[i], data[i+1], label[i+1], n_components, n_features, epsilon, percent, k)
            pred = np.array(pred, dtype=int)
            # print(label[i])
            pred = pred.flatten()

            # print(pred)
            acc, f1, mcc = metrics.makeBatches(label[i], pred, len(label[i]), step)

            predicted = np.vstack([predicted, pred])
            # print(predicted)
            # sys.exit(0)

            updt += upd
            acc_percent = np.vstack([acc_percent, acc])
            f1_percent = np.vstack([f1_percent, f1])
            mcc_percent = np.vstack([mcc_percent, mcc])


        # print(acc_percent)

        predicted = predicted.flatten()
        lab = label_[len(label[0]):]


        score = accuracy_score(lab, predicted)
        f1 = f1_score(lab, predicted, average = 'macro')
        mcc = matthews_corrcoef(lab, predicted)
        # score, f1, mcc = metrics.metrics(acc_percent, lab, predicted, step, f1_type = 'macro')
        # print(a,score, f1, mcc)
    return score, f1, mcc


In [3]:
poolsize = 300
clusters = 2
n_components = 4
# episilon = 0.15
step = 5

In [4]:
base = '/home/god/Documentos/tcc/datasets/'
# list = ['NOAA.txt', 'elec.txt']#, 'keystroke.txt']
list = ['keystroke.txt']
# list = ['NOAA.txt']#, 'elec.txt']
database = {}

for i in range(0, len(list)):
    database[i] = base + list[i]

array_ep = [0.05, 0.10, 0.15]
# array_ep = [0.1]
array_p = [10, 20, 30]
# array_p = [30]
array_k = [50, 25, 12, 7, 5, 3, 1]

result = [0.0, 0.0, 0.0]
aux = []
result = np.array(result, dtype=float)

dic = {}

In [5]:
dic = {}
for key, value in database.items():
    dic[int(key)+5] = {}
    for ep in range(0, len(array_ep)):
        for p in range(0, len(array_p)):
            for k in range(0, len(array_k)):

                adr = value
                dataset, data_labeled, dataset_train, l_train, stream, l_stream, n_features = u.criar_datasets(5, adr)

                score, f1, mcc = doTrain(dataset_train, l_train, step, n_features, array_p[p], array_ep[ep], n_components, array_k[k])

                aux = np.hstack([score, f1, mcc])
                result = np.vstack([result, aux])

                a = str(array_ep[ep])
                b = str(array_p[p])
                c = str(array_k[k])
                name = a +' / '+ b + ' / ' + c
                dic[int(key)+5][name] = aux

            
for p_id, p_info in dic.items():
    print('dataset :', p_id)
    for key in p_info:
        print(key, ':', p_info[key])

/home/god/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/god/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:538: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/god/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


dataset : 5
0.05 / 10 / 50 : [0. 0. 0.]
0.05 / 10 / 25 : [0. 0. 0.]
0.05 / 10 / 12 : [0. 0. 0.]
0.05 / 10 / 7 : [0. 0. 0.]
0.05 / 10 / 5 : [0. 0. 0.]
0.05 / 10 / 3 : [0.25       0.10416667 0.        ]
0.05 / 10 / 1 : [0.25       0.18371212 0.        ]
0.05 / 20 / 50 : [0. 0. 0.]
0.05 / 20 / 25 : [0. 0. 0.]
0.05 / 20 / 12 : [0. 0. 0.]
0.05 / 20 / 7 : [ 0.2         0.08510638 -0.15361907]
0.05 / 20 / 5 : [ 0.225       0.16023216 -0.04761905]
0.05 / 20 / 3 : [0.325      0.28921569 0.1196654 ]
0.05 / 20 / 1 : [0.275      0.22594086 0.03770231]
0.05 / 30 / 50 : [0. 0. 0.]
0.05 / 30 / 25 : [0. 0. 0.]
0.05 / 30 / 12 : [0.25 0.1  0.  ]
0.05 / 30 / 7 : [0.325      0.2347561  0.14165765]
0.05 / 30 / 5 : [ 0.125       0.09109312 -0.2220578 ]
0.05 / 30 / 3 : [0.4        0.29487179 0.26375219]
0.05 / 30 / 1 : [0.275      0.22594086 0.03770231]
0.1 / 10 / 50 : [0. 0. 0.]
0.1 / 10 / 25 : [0. 0. 0.]
0.1 / 10 / 12 : [0. 0. 0.]
0.1 / 10 / 7 : [0. 0. 0.]
0.1 / 10 / 5 : [0. 0. 0.]
0.1 / 10 / 3 : [0.25    